In [2]:
## Load libraries

import pandas as pd
import numpy as np
import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.preprocessing import MultiLabelBinarizer


%load_ext autoreload
%autoreload 2

In [3]:
og_glove_file = r"/Users/anshuman/UMass/CS 682/Project/glove.6B/glove.6b.300d.txt" ## glove text file that you download from internet, 6b words 300d,
new_glove_file =r"glove6b300d.bin"
movie_data_file =r"../ml-1m/movies.dat" #movie data file
user_data_file =r"../ml-1m/users.dat" #user data file
rating_data_file = r"../ml-1m/ratings.dat" #rating data file

In [4]:
##Run this only once ever, if you have skip this and go to the next cell
## Insert your glove location below.
## This is done to convert .txt glove to binary format for easier loading.
# glove_file = datapath(og_glove_file)
# tmp_file = get_tmpfile("glove.txt")
# _ = glove2word2vec(glove_file, tmp_file)
# model = KeyedVectors.load_word2vec_format(tmp_file)
# model.save_word2vec_format(new_glove_file,binary=True)

In [5]:
 ##load binary glove model
glove_model = gensim.models.KeyedVectors.load_word2vec_format(new_glove_file,binary=True)
##get vocabulary
vocab = list(glove_model.wv.vocab.keys())

/Users/anshuman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


In [6]:
## loading the dataset into dataframes

movieData = pd.read_csv(movie_data_file,sep="::",names=["MovieID","Name","Genres"],engine='python')
userData = pd.read_csv(user_data_file,sep="::",names=["UserID","Gender","Age","Occupation","Zipcode"],engine='python')
ratingData = pd.read_csv(rating_data_file,sep="::",names=["UserID","MovieID","Rating","Timestamp"],engine='python')


In [7]:
# ratingData['MovieID'].unique().shape

# user_id = ratingData[['UserID']].drop_duplicates().reindex()
# user_id['userId'] = np.arange(len(user_id))
# ratingData = pd.merge(ratingData, user_id, on=['UserID'], how='left')
item_id = ratingData[['MovieID']].drop_duplicates()
item_id['itemId'] = np.arange(len(item_id))
ratingData = pd.merge(ratingData, item_id, on=['MovieID'], how='left')
# ratingData = ratingData[['userId', 'itemId', 'Rating', 'Timestamp']]
# print('Range of userId is [{}, {}]'.format(ratingData.userId.min(), ratingData.userId.max()))
# print('Range of itemId is [{}, {}]'.format(ratingData.itemId.min(), ratingData.itemId.max()))

In [8]:
print (item_id.iloc[14])

MovieID    1035
itemId       14
Name: 14, dtype: int64


In [9]:
## movie data preprocessing  - getting a dictionary of movie name and id pairs,
##creating genrelist, and reseting movieData dataframe

movieNameData = movieData["Name"].unique()
movieNameDict = {movieNameData[i]:i for i in range(0,movieNameData.shape[0])}
genreList = ["Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir",
              "Horror","Musical","Mystery","Romance","Sci-Fi","Thriller","War","Western"]
genres = movieData["Genres"].apply(lambda x : x.split("|"))
movieData.reset_index(inplace=True,drop=True)

In [10]:
from data_preparation import MovieEncodingEmbedding

print(movieData.shape)
numRows = movieData.shape[0]
# print(movieData, movieData.loc[0][0], movieData.loc[0][1], movieData.loc[0][2])
movie_embeddings = np.zeros((numRows, 319))

# Prepare movie embeddings -> A Mapping from MovieID to its embedding. This can then be used during model training
for rowNum in range(numRows):
    movieID = movieData.iloc[rowNum]['MovieID']
#     print(rowNum, movieID)
    movie_embeddings[rowNum] = MovieEncodingEmbedding(movieID, movieData, genreList, glove_model, vocab)

print("Movie Embeddings", movie_embeddings.shape)

(3883, 3)
Movie Embeddings (3883, 319)


In [11]:
from data_preparation import UserSequences, GetAllUserSequences

## get user Sequences
userSequences = UserSequences(ratingData,-1)

allUserSequences = GetAllUserSequences(ratingData, 20)

27


In [12]:
print(len(userSequences))
print(len(userSequences[0]))
print(userSequences[99][1])

print(len(allUserSequences))
print(len(allUserSequences[0]))
print(allUserSequences[99][1])

5421
27
(1753,)
47316
20
(1034,)


In [13]:
allUserSequences[123:1235]

[[(849,),
  (503,),
  (1198,),
  (133,),
  (86,),
  (66,),
  (1202,),
  (1215,),
  (1190,),
  (1209,),
  (327,),
  (1207,),
  (1227,),
  (1175,),
  (1171,),
  (1139,),
  (561,),
  (182,),
  (687,),
  (1205,)],
 [(673,),
  (537,),
  (746,),
  (22,),
  (1120,),
  (273,),
  (15,),
  (701,),
  (625,),
  (614,),
  (642,),
  (1210,),
  (19,),
  (424,),
  (688,),
  (385,),
  (198,),
  (1176,),
  (730,),
  (1185,)],
 [(192,),
  (715,),
  (577,),
  (635,),
  (864,),
  (1212,),
  (207,),
  (200,),
  (1218,),
  (1211,),
  (1216,),
  (131,),
  (1230,),
  (592,),
  (1191,),
  (1222,),
  (772,),
  (604,),
  (116,),
  (350,)],
 [(290,),
  (221,),
  (335,),
  (556,),
  (1224,),
  (179,),
  (138,),
  (515,),
  (890,),
  (371,),
  (769,),
  (1150,),
  (20,),
  (552,),
  (1208,),
  (732,),
  (83,),
  (189,),
  (791,),
  (148,)],
 [(534,),
  (343,),
  (1132,),
  (89,),
  (1200,),
  (1186,),
  (446,),
  (618,),
  (1181,),
  (740,),
  (647,),
  (1165,),
  (793,),
  (828,),
  (862,),
  (274,),
  (657,),
  (1

In [14]:
## get user-movie encodings for the data with embedding flag false and write it to file.
## In every file each line will contain
##userid::gender::userDataAge::userDataOccupation::userDataZipCode::movieid::(one hot genre encoding of len 18[each separated by ::]):rating 
# data = UserMoviePairEncodingBatch(userSequences,False)

# with open("usermovie.dat","w") as f:
#     for i in range(len(data)):
#         for j in range(data[i].shape[0]):
#             f.write("::".join(data[i][j]))
#             f.write("\n")
                 

In [15]:
## get user-movie encodings for the data with embedding flag false and write it to file.
## In every file each line will contain
##userid::gender::userDataAge::userDataOccupation::userDataZipCode::(movieid_embedding[len 300, separated by ::])::(one hot genre encoding of len 18[each separated by ::]):rating 

# data = UserMoviePairEncodingBatch(userSequences,True)

# with open("usermoviembeddings.dat","w") as f:
#     for i in range(len(data)):
#         for j in range(data[i].shape[0]):
#             f.write("::".join(data[i][j]))
#             f.write("\n")

In [16]:
# np.array(data).shape
# print ("data", len(data), data[0].shape)
from sklearn.model_selection import train_test_split

def fix(data):
    newData = np.empty(data.shape, dtype=int)
    for i, userSeq in enumerate(data):
        for j, tup in enumerate(userSeq):
            newData[i][j] = tup[0]
    return newData

# data = fix(np.array(userSequences))
data = fix(np.array(allUserSequences))
train_data, test_data = train_test_split(data, test_size=0.2)
train_data, val_data = train_test_split(train_data, test_size=0.2)

train_inp = movie_embeddings[train_data[:, :-1]]
train_tgt = train_data[:,1:]
print ("Train tgt", np.amax(train_tgt, 1))

test_inp = movie_embeddings[test_data[:, :-1]]
test_tgt = test_data[:, 1:]

val_inp = movie_embeddings[val_data[:, :-1]]
val_tgt = val_data[:, 1:]

print ("train", train_data.shape, train_inp.shape, train_tgt.shape)
print ("val", val_data.shape, val_inp.shape, val_tgt.shape)
print ("test", test_data.shape, test_inp.shape, test_tgt.shape)            

Train tgt [2308 3251 2784 ... 1913 2769 2526]
train (30281, 20) (30281, 19, 319) (30281, 19)
val (7571, 20) (7571, 19, 319) (7571, 19)
test (9464, 20) (9464, 19, 319) (9464, 19)


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [18]:
def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

In [19]:
# train_data = batchify(torch.Tensor(train_data), 10)
# val_data = batchify(torch.Tensor(val_data), 10)
# test_data = batchify(torch.Tensor(test_data), 10)

train_data_final = torch.Tensor(train_inp)
val_data_final = torch.Tensor(val_inp)
test_data_final = torch.Tensor(test_inp)

train_tgt_final = torch.Tensor(train_tgt).long()
val_tgt_final = torch.Tensor(val_tgt).long()
test_tgt_final = torch.Tensor(test_tgt).long()

print("Train", train_data_final.shape, train_tgt_final.shape)
print("Val", val_data_final.shape, val_tgt_final.shape)
print("Test", test_data_final.shape, test_tgt_final.shape)

Train torch.Size([30281, 19, 319]) torch.Size([30281, 19])
Val torch.Size([7571, 19, 319]) torch.Size([7571, 19])
Test torch.Size([9464, 19, 319]) torch.Size([9464, 19])


In [ ]:
import time
import math
from model import RNNModel
from learner import Learner

device = "cpu"

_, temporal_batch_size, input_size = train_data_final.shape
ntokens = 3706
nlayers = 3
hidden_size = 128
dropout = 0.7
train_batch_size = 256
val_batch_size = val_data_final.shape[0]
test_batch_size = test_data_final.shape[0]

model = RNNModel('gru', ntokens, input_size, hidden_size, nlayers, dropout)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=5)
optimizer = optim.SGD(model.parameters(), lr=10)
learner = Learner(model, criterion, optimizer)

best_val_loss = None
num_epochs = 30

# Loop over epochs.
for epoch in range(1, num_epochs+1):
    print ("| epoch {:3d} |".format(epoch))
    epoch_start_time = time.time()
    learner.train(train_data_final, train_tgt_final, ntokens, train_batch_size)
    val_loss = learner.evaluate(val_data_final, val_tgt_final, ntokens, val_batch_size)
    generated_seq = learner.generate_seq (3000, movie_embeddings, 5)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | val loss {:5.2f} '
          .format(epoch, (time.time() - epoch_start_time),
                                       val_loss))
    print('-' * 89)
    print("Generated Sequence:")
    for id in generated_seq:
        movie = movieData.loc[movieData["MovieID"]==id]
        print(movie.to_string(columns=['Name', 'Genres'], index=False, header=False))
    print('-' * 89)
    # Save the model if the validation loss is the best we've seen so far.
    if not best_val_loss or val_loss < best_val_loss:
        with open("../checkpoints/seq_3_128_0.7_256_20_best.pt", 'wb') as f:
            torch.save(model.state_dict(), f)
        best_val_loss = val_loss
#     else if lr > 1e-3:
        # Anneal the learning rate if no improvement has been seen in the validation dataset.
#         lr /= 1.5

| epoch   1 |
Num Batches 119
    5/  119 batches  | loss  8.18
   10/  119 batches  | loss  8.10
   15/  119 batches  | loss  8.02
   20/  119 batches  | loss  7.96
   25/  119 batches  | loss  7.91
   30/  119 batches  | loss  7.88
   35/  119 batches  | loss  7.85
   40/  119 batches  | loss  7.82
   45/  119 batches  | loss  7.81
   50/  119 batches  | loss  7.79
   55/  119 batches  | loss  7.77
   60/  119 batches  | loss  7.76
   65/  119 batches  | loss  7.75
   70/  119 batches  | loss  7.74
   75/  119 batches  | loss  7.73
   80/  119 batches  | loss  7.72
   85/  119 batches  | loss  7.71
   90/  119 batches  | loss  7.70
   95/  119 batches  | loss  7.70
  100/  119 batches  | loss  7.69
  105/  119 batches  | loss  7.69
  110/  119 batches  | loss  7.68
  115/  119 batches  | loss  7.68
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 92.32s | val loss  7.56 
----------------------------------------------

    5/  119 batches  | loss  7.54
   10/  119 batches  | loss  7.53
   15/  119 batches  | loss  7.53
   20/  119 batches  | loss  7.52
   25/  119 batches  | loss  7.52
   30/  119 batches  | loss  7.52
   35/  119 batches  | loss  7.53
   40/  119 batches  | loss  7.53
   45/  119 batches  | loss  7.53
   50/  119 batches  | loss  7.53
   55/  119 batches  | loss  7.53
   60/  119 batches  | loss  7.53
   65/  119 batches  | loss  7.53
   70/  119 batches  | loss  7.53
   75/  119 batches  | loss  7.53
   80/  119 batches  | loss  7.52
   85/  119 batches  | loss  7.52
   90/  119 batches  | loss  7.52
   95/  119 batches  | loss  7.52
  100/  119 batches  | loss  7.52
  105/  119 batches  | loss  7.52
  110/  119 batches  | loss  7.52
  115/  119 batches  | loss  7.52
-----------------------------------------------------------------------------------------
| end of epoch   7 | time: 70.53s | val loss  7.53 
----------------------------------------------------------------------------

    5/  119 batches  | loss  7.53
   10/  119 batches  | loss  7.52
   15/  119 batches  | loss  7.52
   20/  119 batches  | loss  7.51
   25/  119 batches  | loss  7.51
   30/  119 batches  | loss  7.51
   35/  119 batches  | loss  7.52
   40/  119 batches  | loss  7.52
   45/  119 batches  | loss  7.52
   50/  119 batches  | loss  7.52
   55/  119 batches  | loss  7.52
   60/  119 batches  | loss  7.52
   65/  119 batches  | loss  7.52
   70/  119 batches  | loss  7.52
   75/  119 batches  | loss  7.52
   80/  119 batches  | loss  7.51
   85/  119 batches  | loss  7.51
   90/  119 batches  | loss  7.51
   95/  119 batches  | loss  7.51
  100/  119 batches  | loss  7.51
  105/  119 batches  | loss  7.51
  110/  119 batches  | loss  7.51
  115/  119 batches  | loss  7.51
-----------------------------------------------------------------------------------------
| end of epoch  13 | time: 65.48s | val loss  7.52 
----------------------------------------------------------------------------

    5/  119 batches  | loss  7.52
   10/  119 batches  | loss  7.51
   15/  119 batches  | loss  7.51
   20/  119 batches  | loss  7.50
   25/  119 batches  | loss  7.50
   30/  119 batches  | loss  7.51
   35/  119 batches  | loss  7.51
   40/  119 batches  | loss  7.51
   45/  119 batches  | loss  7.51
   50/  119 batches  | loss  7.51
   55/  119 batches  | loss  7.51
   60/  119 batches  | loss  7.51
   65/  119 batches  | loss  7.51
   70/  119 batches  | loss  7.51
   75/  119 batches  | loss  7.51
   80/  119 batches  | loss  7.51
   85/  119 batches  | loss  7.51
   90/  119 batches  | loss  7.50
   95/  119 batches  | loss  7.51
  100/  119 batches  | loss  7.51
  105/  119 batches  | loss  7.51
  110/  119 batches  | loss  7.51
  115/  119 batches  | loss  7.50
-----------------------------------------------------------------------------------------
| end of epoch  19 | time: 66.59s | val loss  7.52 
----------------------------------------------------------------------------

In [ ]:
learner.plotLearningCurve()

In [ ]:
test_model = RNNModel('gru', ntokens, input_size, hidden_size, nlayers, dropout)
test_model.load_state_dict(torch.load("model.pt"))
test_model.eval()
generator = Learner(test_model, criterion, optimizer)

In [ ]:
generated_seq = learner.generate_seq (3000, movie_embeddings, 10)
print("Generated Sequence:")
for id in generated_seq:
    movie = movieData.loc[movieData["MovieID"]==id]
    print(movie.to_string(columns=['Name', 'Genres'], index=False, header=False))

In [ ]:
test_loss = learner.evaluate(test_data_final, test_tgt_final, ntokens, test_batch_size)
print("Test Loss: ", test_loss)

In [ ]:
from data_preparation import MovieEncodingEmbedding

MovieEncodingEmbedding(29, movieData, genreList, glove_model, vocab)